In [4]:
### Imports ###

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib 
from matplotlib.backends.backend_pdf import PdfPages
import math as m 
import os
import sys
import re

from astropy.utils import data
from astropy.wcs import WCS
from astropy.io import fits
from spectral_cube import SpectralCube
from astropy import units as u
from astropy.modeling import models, fitting
from astropy.io import ascii
from astropy.table import Table

import warnings
warnings.filterwarnings('ignore')

# Set path to your home directory. Path will be used for all functions unless otherwise specified
path='/Users/jane/Desktop'   # path 

In [13]:
# directory where the data are located and the files are to be written
datadir = '/Users/jane/Desktop/'

# name of the input temperature map/FITS file
temp_fitsfilename = 's138_temp_regrid.fits'

# Name of the spectral data cubes on which to perform gaussian fitting
hco_spec_filename='w40_hco+_smooth_gal.fits'       # input fits file name
hcn_spec_filename='w40_hcn_smooth_gal.fits'        # input fits file name
#hco_spec_filename='s138_HCO+_smooth.fits'       # input fits file name
#hcn_spec_filename='s138_HCN_smooth.fits'        # input fits file name

# path to the radex molecular data files
radexpath = '/Applications/CASSIS/database/lamda/'

# extension to the molecular data files
extension = '.dat'

# path to the radex executable program
radexexec = '/Users/plume/Source/Radex/bin/'

##################################################

# Parameters for the Gaussian fitting.  Change them for each new source

vel_guess = 7.5  # The initial guess for the centroid velocity of the gaussian fit
peak_guess = 8   # the initial guess for the maximum line strength (sets the yscaling of the plots)
#vel_guess = -52   # The initial guess for the centroid velocity of the gaussian fit
#peak_guess = 4    # the initial guess for the maximum line strength (sets the yscaling of the plots)
dv_min = 1        # set a minimum acceptable line width
dv_max = 10       # set a maximum acceptable line width
snr_min = 3.0     # set a minumum acceptable SNR

# set the range of pixels in the x direction over which to loop
#min_xpix = 0      # s138 all
#max_xpix = 49     # s138 all
#min_xpix = 2     # s138 small
#max_xpix = 11    # s138 small
min_xpix = 29    # w40
max_xpix = 56    # w40

# set the range of pixels in the y direction over which to loop
#min_ypix = 0      # s138 all
#max_ypix = 46     # s138 all
#min_ypix = 38    # s138 small
#max_ypix = 46    # s138 small
min_ypix = 15    # w40
max_ypix = 50    # w40

#################################################

# density for the RADEX models
nh2 = 1.0e5      # nH2 cm^-3

##################################################

# Names of various output files.  YOU ONLY NEED TO CHANGE THESE IF YOU WANT
# DIFFERENT NAMES THAN THE DEFAULT ONES

# name of the output ascii text file containing the dust temperature
temp_asciifile = 'temp.txt'

# Name the name of the output ascii files with the Gaussian fits
hco_gauss_outfile = 'hco+_gaussfitsTEST2.txt' # file name of the gaussian fit results
hcn_gauss_outfile = 'hcn_gaussfitsTEST2.txt' # file name of the gaussian fit results

# Name of the RADEX input files that the code creates
radexhcoinp = 'radexHCO+.inp'
radexhcninp = 'radexHCN.inp'

# Name of the RADEX output files
radexhcoout = 'radexHCO+.out'
radexhcnout = 'radexHCN.out'

# Name of the output files which contain position and column density for all pixels
hcocolden   = 'HCO+_colden.txt'
hcncolden   = 'HCN_colden.txt'
coldenratio = 'ColDenRatio.txt'

In [14]:
warnings.filterwarnings('ignore') # Turn warnings off (because they're annoying!)

#JCMT efficiency at 230 GHz to convert TA* to Tmb
eta = 0.6

#### Fit HCO+
# retrieving the source data and information
datfile = fits.open(datadir+hco_spec_filename)
cube = SpectralCube.read(datfile)
wcs = WCS(datfile[0].header) # vel and position info from WCS in header

# extract the world coordinates of all the pixels using the world property, 
# which returns the spectral axis then the two positional coordinates in reverse order 
# (in the same order as the data indices - i.e. spectral axis, Y, X). 
velo, lat, long  = cube.world[:] 

# convert the velocity units from m/s (in the fits header) to km/s
cube2 = cube.with_spectral_unit(u.km / u.s)

# Make a sub-cube using a part of the spectrum that is free of spectral line emission 
# This will be used to calculate the noise in each spectrum
linefree = cube2.spectral_slab((vel_guess*u.km / u.s) + (20*u.km / u.s),(vel_guess*u.km / u.s) + (100*u.km / u.s))

# print column headers to screen
#print("HCO+")
#print("x  y          Glat.                 Glong.           TA   Tmb    V     Sig   Fwhm")
#print("                                                      K    K    km/s   km/s  km/s")
        
# initialize the arrays to write to an ascii table file at the end
x = []
y = []
glat = []
glong = []
vlsr = []
TA = []
sigma = []
fwhm = []
TAdv = []
Tmb = []
Tmbdv = []
figs = []

# set an index for the array entries
n = 0

# Loop over all pixels in the x and y ranges provided, extract the spectrum at that pixel,
# fit a gaussian to the spectrum and print and plot the results
for i in range(min_xpix,max_xpix+1):
    for j in range(min_ypix,max_ypix+1):
        # grab a spectrum at pixel x, y 
        x.append(i)
        y.append(j)
        T = cube2[:, j, i]               # get the intensity/temperature along the spectral axis
        vel = cube2.spectral_axis        # set the spectral/velocity axis
        glat.append(lat[0,j,0])          # get the DEC/lat of pixel x/y from the world coordinates
        glong.append(long[0,0,i])        # get the RA/long of pixel x/y from the world coordinates

        # Fit the selected spectrum at pixel x,y using a Gaussian
        g_init = models.Gaussian1D(amplitude=1.0, mean=vel_guess, stddev=1.0)
        fit_g = fitting.LevMarLSQFitter()
        g = fit_g(g_init, vel, T)
        
        gT = g.amplitude.value   # Line peak temperature of the gaussian fit  (K)
        gV = g.mean.value        # line centre velocity of the gaussian fit (km/s)
        gS = g.stddev.value      # standard deviation (sigma) width of the gaussian fit (km/s)
        
        # calculate the noise in the spectrum at this position
        noiseT = linefree[:,j,i]
        noise = noiseT.std()
        #print(noise,gT*u.K,gT*u.K/noise)
               
        # If the Gaussian fit intensity is >  SNR * noise
        # and the FWHM is > some minimum width and < some maximum width
        # then accept the values of the gaussian fits
        # if not, then we assume that the spectrum is weak, the fit is nonsense and so
        # we set the values to 0 so we know what to ignore in the table
        if gT*u.K > (snr_min*noise) and (2.355*gS) > dv_min and (2.355*gS) < dv_max:
         
            TA.append(gT)               # Line peak temperature of the gaussian fit  (K)
            vlsr.append(gV)             # line centre velocity of the gaussian fit (km/s)
            sigma.append(gS)            # standard deviation (sigma) width of the gaussian fit (km/s)
            fwhm.append(2.355 * gS)     # Full Width at Half Maximum of the gaussian fit  (km/s)
            Tmb.append(gT/eta)          # Calibrated Line peak temperature of the gaussian fit  (K)
            flag = "included"           # print a flag on the plot to indicate this fit is good
        else:
            TA.append(0)           # Line peak temperature of the gaussian fit  (K)
            vlsr.append(0)         # line centre velocity of the gaussian fit (km/s)
            sigma.append(0)        # standard deviation (sigma) width of the gaussian fit (km/s)
            fwhm.append(0)         # Full Width at Half Maximum of the gaussian fit  (km/s)
            Tmb.append(0)          # Calibrated Line peak temperature of the gaussian fit  (K)
            flag = "ignored"       # print a flag on the plot to indicate this fit is ignored
        
        # print the fit results to the screen
        #print(x[n], y[n], glat[n], glong[n], "%5.2f"%TA[n], "%5.2f"%Tmb[n], "%5.2f"%vlsr[n],"%5.2f"%sigma[n], "%5.2f"%fwhm[n]) 
        
        # plot each of the spectra and fits    
        fig = plt.figure(figsize=(10,7), facecolor='white')
        ax = fig.add_subplot(111)  # make a single panel subplot
        im = ax.step(vel, T, where='mid', label='HCO+ Data') # plot the data as a histogram
        im = ax.plot(vel, g(vel), label='Gaussian')     # plot the gaussian fit
        ax.set_xlim([vel_guess-20,vel_guess+20])
        ax.set_ylim([-1,peak_guess])
        ax.set_xlabel("V (km/s)", fontsize=16)
        ax.set_ylabel("T$_A^*$ (K)", fontsize=16)
        ax.text(vel_guess+15,peak_guess-1.5,i)  # y pixel
        ax.text(vel_guess+16.5,peak_guess-1.5,j) # x pixel
        ax.text(vel_guess+15,peak_guess-2.5,flag) # plot the flag
        ax.legend()
        figs.append(fig) # Figure of Gaussian fit to be saved 
        plt.close(fig) 
        
        n += 1    #increment the array index



# Set the header line, the parameters, and the format of the parameters to 
# write to an ascii table file        
out = Table()
out['Pix_x'] = x
out['Pix_y'] = y
out['GLat(deg)'] = glat
out['GLat(deg)'].info.format = '8.5f'
out['GLong(deg)'] = glong
out['GLong(deg)'].info.format = '8.5f'
out['TA(K)'] = TA
out['TA(K)'].info.format = '6.2f'
out['Tmb(K)'] = Tmb
out['Tmb(K)'].info.format = '6.2f'
out['VLSR(km/s)'] = vlsr
out['VLSR(km/s)'].info.format = '6.2f'
out['sigma(km/s)'] = sigma
out['sigma(km/s)'].info.format = '6.2f'
out['FWHM(km/s)'] = fwhm
out['FWHM(km/s)'].info.format = '6.2f'
#out['TAdv(K km/s)'] = TAdv
#out['TAdv(K km/s)'].info.format = '6.2f'
#out['Tmbdv(K km/s)'] = Tmbdv
#out['Tmbdv(K km/s)'].info.format = '6.2f'

# Write the gaussian fits of all pixels to a text file 
ascii.write(out, datadir+hco_gauss_outfile,  overwrite=True, format='tab')

# saving the list containing all the plots to a pdf file
def save_multi_image(filename):
    pp = PdfPages(filename)
    for fig in figs:
        fig.savefig(pp, format='pdf')
    pp.close()
    
# path where pdf file will be saved
pdffile = datadir+"hco+_multiTEST.pdf"
#save_multi_image(pdffile) # Comment out if you do not want plots saved to PDF
print('For HCO+:')
print('Gaussian fit results written to',datadir+hco_gauss_outfile)
print ('Gaussian fit plots written to', pdffile) 



#### Fit HCN
# retrieving the source data and information
datfile = fits.open(datadir+hcn_spec_filename)
cube = SpectralCube.read(datfile)
wcs = WCS(datfile[0].header) # vel and position info from WCS in header

# extract the world coordinates of all the pixels using the world property, 
# which returns the spectral axis then the two positional coordinates in reverse order 
# (in the same order as the data indices - i.e. spectral axis, Y, X). 
velo, lat, long  = cube.world[:] 

# convert the velocity units from m/s (in the fits header) to km/s
cube2 = cube.with_spectral_unit(u.km / u.s)

# Make a sub-cube using a part of the spectrum that is free of spectral line emission 
# This will be used to calculate the noise in each spectrum
linefree = cube2.spectral_slab((vel_guess*u.km / u.s) + (20*u.km / u.s),(vel_guess*u.km / u.s) + (100*u.km / u.s))

# print column headers to screen
#print("HCN")
#print("x  y          Glat.                 Glong.           TA   Tmb    V     Sig   Fwhm")
#print("                                                      K    K    km/s   km/s  km/s")
        
# initialize the arrays to write to an ascii table file at the end
x = []
y = []
glat = []
glong = []
vlsr = []
TA = []
sigma = []
fwhm = []
TAdv = []
Tmb = []
Tmbdv = []
figs = []

# set an index for the array entries
n = 0

# Loop over all pixels in the x and y ranges provided, extract the spectrum at that pixel,
# fit a gaussian to the spectrum and print and plot the results
for i in range(min_xpix,max_xpix+1):
    for j in range(min_ypix,max_ypix+1):
        # grab a spectrum at pixel x, y 
        x.append(i)
        y.append(j)
        T = cube2[:, j, i]               # get the intensity/temperature along the spectral axis
        vel = cube2.spectral_axis        # set the spectral/velocity axis
        glat.append(lat[0,j,0])          # get the DEC/lat of pixel x/y from the world coordinates
        glong.append(long[0,0,i])        # get the RA/long of pixel x/y from the world coordinates

        # Fit the selected spectrum at pixel x,y using a Gaussian
        g_init = models.Gaussian1D(amplitude=1.0, mean=vel_guess, stddev=1.0)
        fit_g = fitting.LevMarLSQFitter()
        g = fit_g(g_init, vel, T)
        
        gT = g.amplitude.value   # Line peak temperature of the gaussian fit  (K)
        gV = g.mean.value        # line centre velocity of the gaussian fit (km/s)
        gS = g.stddev.value      # standard deviation (sigma) width of the gaussian fit (km/s)
        
        # calculate the noise in the spectrum at this position
        noiseT = linefree[:,j,i]
        noise = noiseT.std()
        #print(noise,gT*u.K,gT*u.K/noise)
        
        # If the Gaussian fit intensity is >  SNR * noise
        # and the FWHM is > some minimum width and < some maximum width
        # then accept the values of the gaussian fits
        # if not, then we assume that the spectrum is weak, the fit is nonsense and so
        # we set the values to 0 so we know what to ignore in the table
        if gT*u.K > (snr_min*noise) and (2.355*gS) > dv_min and (2.355*gS) < dv_max:
            TA.append(gT)               # Line peak temperature of the gaussian fit  (K)
            vlsr.append(gV)             # line centre velocity of the gaussian fit (km/s)
            sigma.append(gS)            # standard deviation (sigma) width of the gaussian fit (km/s)
            fwhm.append(2.355 * gS)     # Full Width at Half Maximum of the gaussian fit  (km/s)
            Tmb.append(gT/eta)          # Calibrated Line peak temperature of the gaussian fit  (K)
            flag = "included"           # print a flag on the plot to indicate this fit is good
        else:
            TA.append(0)           # Line peak temperature of the gaussian fit  (K)
            vlsr.append(0)         # line centre velocity of the gaussian fit (km/s)
            sigma.append(0)        # standard deviation (sigma) width of the gaussian fit (km/s)
            fwhm.append(0)         # Full Width at Half Maximum of the gaussian fit  (km/s)
            Tmb.append(0)          # Calibrated Line peak temperature of the gaussian fit  (K)
            flag = "ignored"       # print a flag on the plot to indicate this fit is ignored
        
        # print the fit results to the screen
        #print(x[n], y[n], glat[n], glong[n], "%5.2f"%TA[n], "%5.2f"%Tmb[n], "%5.2f"%vlsr[n],"%5.2f"%sigma[n], "%5.2f"%fwhm[n]) 
        
        # plot each of the spectra and fits    
        fig = plt.figure(figsize=(10,7), facecolor='white')
        ax = fig.add_subplot(111)  # make a single panel subplot
        im = ax.step(vel, T, where='mid', label='HCN Data') # plot the data as a histogram
        im = ax.plot(vel, g(vel), label='Gaussian')     # plot the gaussian fit
        ax.set_xlim([vel_guess-20,vel_guess+20])
        ax.set_ylim([-1,peak_guess])
        ax.set_xlabel("V (km/s)", fontsize=16)
        ax.set_ylabel("T$_A^*$ (K)", fontsize=16)
        ax.text(vel_guess+15,peak_guess-1.5,i)  # y pixel
        ax.text(vel_guess+16.5,peak_guess-1.5,j) # x pixel
        ax.text(vel_guess+15,peak_guess-2.5,flag) # plot the flag
        ax.legend()
        figs.append(fig)
        plt.close(fig) 
        
        n += 1    #increment the array index

# Set the header line, the parameters, and the format of the parameters to 
# write to an ascii table file        
out = Table()
out['Pix_x'] = x
out['Pix_y'] = y
out['GLat(deg)'] = glat
out['GLat(deg)'].info.format = '8.5f'
out['GLong(deg)'] = glong
out['GLong(deg)'].info.format = '8.5f'
out['TA(K)'] = TA
out['TA(K)'].info.format = '6.2f'
out['Tmb(K)'] = Tmb
out['Tmb(K)'].info.format = '6.2f'
out['VLSR(km/s)'] = vlsr
out['VLSR(km/s)'].info.format = '6.2f'
out['sigma(km/s)'] = sigma
out['sigma(km/s)'].info.format = '6.2f'
out['FWHM(km/s)'] = fwhm
out['FWHM(km/s)'].info.format = '6.2f'
#out['TAdv(K km/s)'] = TAdv
#out['TAdv(K km/s)'].info.format = '6.2f'
#out['Tmbdv(K km/s)'] = Tmbdv
#out['Tmbdv(K km/s)'].info.format = '6.2f'

# Write the gaussian fits of all pixels to a text file 
ascii.write(out, datadir+hcn_gauss_outfile,  overwrite=True, format='tab')

# saving the list containing all the plots to a pdf file
def save_multi_image(filename):
    pp = PdfPages(filename)
    for fig in figs:
        fig.savefig(pp, format='pdf')
    pp.close()
    
# path where pdf file will be saved
pdffile = datadir+"hcn_multiTEST.pdf"
#save_multi_image(pdffile) # Comment out if you do not want plots saved to PDF
print('For HCN:')
print('Gaussian fit results written to',datadir+hcn_gauss_outfile)
print ('Gaussian fit plots written to', pdffile) 

# Turn warnings back on
warnings.filterwarnings('default')

For HCO+:
Gaussian fit results written to /Users/jane/Desktop/hco+_gaussfitsTEST2.txt
Gaussian fit plots written to /Users/jane/Desktop/hco+_multiTEST.pdf
For HCN:
Gaussian fit results written to /Users/jane/Desktop/hcn_gaussfitsTEST2.txt
Gaussian fit plots written to /Users/jane/Desktop/hcn_multiTEST.pdf
